# Stroke Key-word extraction

In [24]:
import os
import re

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
raw_dir = '../../data/raw'
readings = pd.read_csv(os.path.join(raw_dir, 'brain_mr_ct_result.csv'), header=None)

In [392]:
df = pd.read_csv(os.path.join(raw_dir, "lab.csv"), parse_dates=['event_time'])

In [364]:
df.head()

,id,case_id,event_type,event_name,event_time,event_result
0,244097,1,LAB,간이혈당검사[POCT],2099-12-30 06:23:00,334
1,244104,1,LAB,간이혈당검사[POCT],2099-12-30 06:23:00,110
2,244107,1,LAB,간이혈당검사[POCT],2099-12-30 06:23:00,180
3,244108,1,LAB,Sodium (serum)(검사24시간가능),2099-12-30 06:23:00,138
4,244109,1,LAB,Potassium (serum)(검사24시간가능),2099-12-30 06:23:00,3.1


In [407]:
cutoff_start = pd.datetime(year=2100, month=1, day=1, hour=0, minute=0,)
cutoff_end = pd.datetime(year=2100, month=4, day=1, hour=0, minute=0,)

is_postop = (df.event_time > cutoff_start) & (df.event_time <= cutoff_end) 
is_img = df_postop.event_type == 'IMG'
is_stroke_related = df.event_name.apply(lambda x: 'CT' in x or 'MR' in x)

df_valid = df[is_postop & is_img & is_stroke_related]
del df

In [205]:
df_valid.head()

,id,case_id,event_type,event_name,event_time,event_result
4123,945,17,IMG,MRI Brain + Brain MRA + Carotid MRA (contrast),2100-01-09 14:24:00,Mild diffuse brain atrophy\nIncreased both lateral and 3rd ventricle size without visible obstructive lesion\nMild degree SVD with a few old lacunar infarct in cerebral WM\nOld ischemic change in Rt prefrontal gyrus\nMRA> \nRt proximal ICA stenosis\nLt proximal
14805,888,90,IMG,MRI Brain + MRA Acute Stroke (contrast),2100-01-04 04:29:00,"S/P Craniotomy and SDH removal, Lt. cerebral convexity.\n \nNo deifnite evidence of diffusion restrictive lesion in the brain.\nMinimal amount of small late subacute extraaxial hemorrhage (T1 and T2 high SI) at Lt occipital convexity.\nA few microbleeds in th"
25097,494,146,IMG,CT Angio + 3D Neck + Brain(contrast),2100-01-01 00:12:00,BRAIN CT AND CTA\n1. No significant focal parenchymal lesion in the brain.\n No significant abnormal finding at CSF space. \n2. No significant focal stenosis or aneurysmal dilatation. \n3. Emphysema and bullae at bilateral upper lungs.
33585,864,216,IMG,MRA+MRI(Brain),2100-01-03 16:17:00,"Outside hospital MR on .\nA small acute infarction in Rt caudate body, and adjacent CR.\n- mild T2 high SI.\n- no hemorrhagic changes.\nSeveral old lacunar infarctions in both BG and Rt thalamus.\nNo abnormal contrast enhancement in brain. \nLt maxillary sinusi"
88683,519,535,IMG,MRI Brain MRA (Aneurysm) (noncontrast),2100-01-07 02:14:00,BRAIN MRA\n1. Approximately 12.5mm sized thrombosed aneurysm at Rt. V4.\n2. Approximately 4.64mm sized fusiform aneurysm at Lt. V4.\n3. Approximately 5.10mm sized extradural unruptured aneurysm at Rt. cavernous ICA.\n4. Approximately 4.24mm sized extradural u


In [408]:
df_valid['event_result'] = df_valid['event_result'].apply(lambda x: x.lower())

# Parse results

In [248]:
s = """ischemi
infarc
high-intensity lesions on DWI-low intensity lesion on ADC
diffusion restriction lesion 
stroke
T2 high signal intensity 
low attenuation-GRE
hemorrhagic transformation
ICH
hemorrhage"""

s = s.lower().split('\n')

In [249]:
s

['ischemi',
 'hemorrhage',
 'infarc',
 'high-intensity lesions on dwi-low intensity lesion on adc',
 'diffusion restriction lesion ',
 'stroke',
 't2 high signal intensity ',
 'low attenuation-gre',
 'hemorrhagic transformation',
 'ich',
 'hemorrhage']

In [304]:
negating_words = ['no', 'old', 'known', 'previous']

In [329]:
keywords = s

def contains_which_keyword(doc):
    has = []
    for kw in keywords:
        if kw in doc:
            if kw == 'stroke':
                if 'brain mri acute stroke' not in doc:
                    has.append(kw)
            else:
                has.append(kw)
    if len(has) == 0:
        return None
    else:
        return has
    
def contains_which_keyword_not_negated(doc):
    doc = doc.split('\n')
    for i, line in enumerate(doc):
        for kw in keywords:
            if kw in line:
                has_any_negated_kw = False
                for nw in negating_words:
                    match = re.search(
                        r'(?<={negation} ).*{keyword}'.format(negation=nw, keyword=kw), 
                        line
                    )
                    has_any_negated_kw |= match is not None

                if not has_any_negated_kw and 'brain mri acute stroke' not in line:
                    return True
    return False
        

def contains_keyword_at_line_number(kw):
    def func(doc):
        doc = doc.split('\n')
        kw = keywords[0]
        for i, line in enumerate(doc):
            if kw in line:
                out_start = re.search(r'(?<!^no) .*{}'.format(kw), line)
                not_negated_at_start = out_start is not None

                out_middle = re.search(r'(?<! no) .*{}'.format(kw), line)
                not_negated_mid_sentence = out_middle is not None

                if not_negated_at_start and not_negated_mid_sentence:
                    return True, i

        return False, None
    return func

In [323]:
nw='no'
kw='infarc'
print(re.search(r'(?<={negation} ).*{keyword}'.format(negation=nw, keyword=kw), 'no evidence of large territorial infarction or hemorrhage').group())
# print(re.search(r'(?<='' ).*{keyword}'.format(negation=nw, keyword=kw), 'no evidence of large territorial infarction or hemorrhage').group())

evidence of large territorial infarc


In [327]:
line = 'no evidence of large territorial infarction or hemorrhage'
for kw in keywords:
    has_any_negated_kw = False
    for nw in negating_words:
        match = re.search(
            r'(?<={negation} ).*{keyword}'.format(negation=nw, keyword=kw), 
            line
        )
        has_any_negated_kw |= match is not None

In [328]:
has_any_negated_kw

True

In [313]:
print(df_valid.event_result.iloc[0])
contains_which_keyword_not_negated(df_valid.event_result.iloc[0])

mild diffuse brain atrophy
increased both lateral and 3rd ventricle size without visible obstructive lesion
mild degree svd with a few old lacunar infarct in cerebral wm
old ischemic change in rt prefrontal gyrus
mra> 
rt proximal ica stenosis
lt proximal


True

In [236]:
del df

In [409]:
df_valid['contains_which_kw'] = df_valid['event_result'].apply(contains_which_keyword)
df_valid['contains_kw_not_negated'] = df_valid['event_result'].apply(contains_which_keyword_not_negated)

In [368]:
df_valid[df_valid.case_id == 1515]

,id,case_id,event_type,event_name,event_time,event_result,contains_which_kw,contains_kw_not_negated
265850,669,1515,IMG,MRI Brain - Hyperacute Stroke (noncontrast),2100-01-07 10:46:00,minimal ivhs (unknown stage) in bilateral lateral ventricles and lt sided 4th ventricle.\nsuspicious minimal sah in rt medial frontal convexity and posterior fossa (along cerebellar follia).\nno visible acute infarction in brain.\nmild t2 hyperintensities in,[infarc],False
265852,674,1515,IMG,CT Routine Brain (Pre contrast),2100-01-08 10:46:00,"no evidence of large territorial infarction or hemorrhage\ndiffuse mild brain atrophy\nmild periventricular hypodensities in bilateral cerebral wm\n(ct에서는 resolution이 제한되어 mri에서 보이는 minimal ivh in bilateral lateral ventricle, 4th ventricle, suspicious minim","[hemorrhage, infarc, hemorrhage]",False
265857,702,1515,IMG,MRI Brain FU Hyperacute Stroke (contrast),2100-01-16 10:46:00,no diffusion restriction lesion\ndisappeared small amount ivh in both lateral ventricle\nno significant change of minimal sah in rt medial frontal convexity and posterior fossa\nmra>\nimproved spastic appearance of intracranial vessels\nno steno-occlusive lesi,None,False


In [410]:
df_valid['contains_kw'] = df_valid['contains_which_kw'].apply(lambda x: x is not None)

In [411]:
df_valid.event_time.max()

Timestamp('2100-03-30 16:19:00')

In [417]:
df_valid.to_csv("../../data/interim/filtered_readings.csv", index=False)

In [413]:
df_valid['contains_which_kw'].apply(lambda x: x is not None).sum()

2153

In [414]:
df_valid['contains_kw'].sum(), len(df_valid[df_valid['contains_kw']].case_id.unique())

(2153, 1058)

In [415]:
df_valid['contains_kw_not_negated'].sum()

1175

In [416]:
len(df_valid[df_valid['contains_kw_not_negated']].case_id.unique())

523

In [375]:
d = df_valid[['case_id', 'contains_kw_not_negated']].groupby('case_id').sum().reset_index().rename(columns={'case_id': 'CaseID'})

In [26]:
clinical = pd.read_csv("../data/raw/clinical.csv")

/Users/suzinyou/anaconda2/envs/datathon/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (20,30,32,36,37,38,40,41,48) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [32]:
clinical.columns

Index(['CaseID', 'Case Start', 'Case End', 'Height', 'Weight', 'BMI',
       'AdmDate', 'DisDate', 'LOS', 'PostOpLOS', 'ICULOS', 'InHospDeath',
       'ASA', 'Emergency', 'Department', 'Diagnosis', 'Operation',
       'Surgical position', 'Anesthesia type', 'Surgery duration',
       'Anesthesia duration', 'Hypertension', 'Diabetes', 'CVA', 'Asthma',
       'COPD', 'LiverDis', 'KidneyDis', 'Tb', 'PreopECG', 'PreopPFT',
       'PreopChest', 'PreopHb', 'PreopPlt', 'PreopAlb', 'PreopBUN', 'PreopCr',
       'PreopNa', 'PreopK', 'PreopPT', 'PreopPTT', 'PreopGlu', 'PreopPH',
       'PreopHCO3', 'PreopBE', 'PreopPaO2', 'PreopPaCO2', 'PreopSpO2',
       'PreopGPT', 'PreopGOT', 'Estimated blood loss (mL)',
       'Urine output (mL)', 'RBC (unit)', 'FFP (unit)', 'Crystalloid (mL)',
       'Colloid (mL)', 'Propofol bolus (mg)', 'Midazolam bolus (mg)',
       'Fentanyl bolus (mcg)', 'Rocuronium (mg)', 'Vecuronium (mg)',
       'Ephedrine (mg)', 'Phenylephrine (mg)', 'Epinephrine (mcg)',
       'Ca

In [34]:
clinical.loc[:, 'CaseID'] = clinical['CaseID'].astype(int)

ValueError: Cannot convert non-finite values (NA or inf) to integer

In [33]:
clinical[['CaseID', 'Department']]

,CaseID,Department
0,1.0,GS
1,2.0,GS
2,3.0,GS
3,4.0,GS
4,5.0,GS
...,...,...
39635,42304.0,GS
39636,42305.0,GS
39637,42306.0,TS
39638,42307.0,TS


In [29]:
clinical.Department

0        GS
1        GS
2        GS
3        GS
4        GS
         ..
39635    GS
39636    GS
39637    TS
39638    TS
39639    GS
Name: Department, Length: 39640, dtype: object

In [359]:
case_ids = clinical.CaseID.dropna()

In [377]:
case_ids = case_ids.astype(int)
labels = case_ids.to_frame()

In [379]:
labels = labels.merge(d, how='outer', on='CaseID').rename(columns={'contains_kw_not_negated': 'label'})

In [381]:
labels = labels.fillna(0)

In [386]:
labels.label = labels.label > 0

In [389]:
labels.label = labels.label.astype(int)

In [390]:
labels.head()

,CaseID,label
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [391]:
labels.to_csv("../../data/processed/labels.csv", index=False)

In [355]:
clinical.merge(d, how='outer', on='CaseID').head(20)

,CaseID,Case Start,Case End,Height,Weight,BMI,AdmDate,DisDate,LOS,PostOpLOS,ICULOS,InHospDeath,ASA,Emergency,Department,Diagnosis,Operation,Surgical position,Anesthesia type,Surgery duration,Anesthesia duration,Hypertension,Diabetes,CVA,Asthma,COPD,LiverDis,KidneyDis,Tb,PreopECG,PreopPFT,PreopChest,PreopHb,PreopPlt,PreopAlb,PreopBUN,PreopCr,PreopNa,PreopK,PreopPT,PreopPTT,PreopGlu,PreopPH,PreopHCO3,PreopBE,PreopPaO2,PreopPaCO2,PreopSpO2,PreopGPT,PreopGOT,Estimated blood loss (mL),Urine output (mL),RBC (unit),FFP (unit),Crystalloid (mL),Colloid (mL),Propofol bolus (mg),Midazolam bolus (mg),Fentanyl bolus (mcg),Rocuronium (mg),Vecuronium (mg),Ephedrine (mg),Phenylephrine (mg),Epinephrine (mcg),Calcium chloride (mg),contains_kw_not_negated
0,1.0,2100-01-01 00:00,2100-01-01 12:12,160.2,65.50,25.522092,2099-12-29 06:23:00,2100-01-08 06:23:00,10.0,8.0,NaN,NaN,2.0,N,GS,Rectal cancer,Low anterior resection,Lithotomy,General,3.033333,3.16667,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Normal,No significant interval change since the last study,14.1,189,4.3,10.0,0.82,141,3.1,94.0,33.2,134,NaN,NaN,NaN,NaN,NaN,NaN,16,18,NaN,300.0,NaN,NaN,350.0,NaN,120.0,NaN,100.0,70.0,NaN,10.0,NaN,NaN,NaN,NaN
1,2.0,2100-01-01 00:00,2100-01-01 13:22,167.3,56.00,20.007653,2099-12-29 10:35:00,2100-01-18 10:35:00,20.0,18.0,NaN,NaN,2.0,N,GS,Advanced gastric cancer,"Subtotal gastrectomy (Billroth II) ( with standard lymph node dissection ) : Diagnostic laparoscopy,, [ONO trial]",Supine,General,3.583333,4.43333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Normal,"Fibrotic lesion in RULF \n\nOtherwise, negative",10.2,251,3.8,14.0,0.86,143,4.7,110.0,31.9,88,NaN,NaN,NaN,NaN,NaN,NaN,15,18,50.0,700.0,NaN,NaN,800.0,NaN,150.0,NaN,NaN,100.0,NaN,20.0,NaN,NaN,NaN,NaN
2,3.0,2100-01-01 00:00,2100-01-01 10:14,169.1,69.70,24.375053,2099-12-29 11:17:00,2100-01-01 11:17:00,3.0,1.0,NaN,NaN,1.0,N,GS,Symptomatic gallbladder stone without obstruction,Cholecystectomy,Reverse Trendelenburg,General,1.250000,1.33333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Normal,"Grossly, no active lung lesion.Normal cardiovascular configuration",14.2,373,4.2,14.0,1.18,144,4.9,103.0,30.3,87,NaN,NaN,NaN,NaN,NaN,NaN,34,17,NaN,NaN,NaN,NaN,200.0,NaN,NaN,NaN,NaN,50.0,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,2100-01-01 00:00,2100-01-01 14:50,160.6,53.70,20.820119,2099-12-29 16:08:00,2100-01-07 16:08:00,9.0,7.0,NaN,NaN,2.0,N,GS,Advanced gastric cancer,TLDG (Totally laparoscopic distal gastrectomy) ( with standard lymph node dissection ) : LDG (open conversion °¡´É),Reverse Trendelenburg,General,4.333333,5.83333,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Normal,Grossly no active lung lesion\n\n,14.4,275,4.1,10.0,0.96,141,4.2,103.0,34.5,108,NaN,NaN,NaN,NaN,NaN,NaN,18,23,NaN,270.0,NaN,NaN,2700.0,NaN,80.0,NaN,100.0,100.0,NaN,50.0,NaN,NaN,NaN,NaN
4,5.0,2100-01-01 00:00,2100-01-01 14:59,171.0,69.50,23.767997,2099-12-31 05:14:00,2100-02-13 05:14:00,44.0,43.0,10.0,NaN,3.0,Y,GS,Aortic aneurysm,"Resection of aneurysm, abdominal aorta + bilateral iliac artery : Aortobiliac bypass, thrombectomyµ¿½Ã ½ÃÇà",Prone,General,5.333333,6.5,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"QTc:448, QRSd:98, I. Conclusion:Normal sinus rhythm Left anterior fascicular block Left ventricular hypertrophy with repolarization abnormality Anteroseptal infarct , age undetermined Abnormal ECG, T:100, PR:176, QRS:-50, P:55, RATE:82, QT:384,",Normal,NaN,15.3,67,2.6,50.0,4.43,146,4.4,73.0,36.5,126,NaN,NaN,NaN,NaN,NaN,NaN,77,765,2600.0,1490.0,8.0,8.0,7100.0,NaN,NaN,NaN,NaN,160.0,NaN,10.0,900.0,NaN,2100.0,NaN
5,6.0,2100-01-01 00:00,2100-01-01 10:26,150.0,54.60,24.266667,2099-12-28 12:31:00,2100-01-02 12:31:00,5.0,2.0,NaN,NaN,2.0,Y,GS,Gallbladder polyp,Cholecystectomy,Supine,General,0.416667,1.33333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Normal,No active lung lesion,12.3,144,3.9,14.0,1.07,141,3.4,104.0,29.1,105,NaN,NaN,NaN,NaN,NaN,NaN,17,19,NaN,NaN,NaN,NaN,100.0,NaN,70.0,NaN,100.0,40.0,NaN,5.0,NaN,NaN,NaN,NaN
6,7.0,2100-01-01 00:00,2100-01-01 13:21,167.5,63.80,22.740031,2099-12-30 05:08:00,2100-01-08 05:0

오름차순으로
case_id, 검사일, 

## 느슨한 기준
* 키워드가 하나라도 들어가있는 느슨한 기준으로 1인 사람들을 위로올린다.
* 그런데 거기 해당하는 case id는 다 보자. 0이어도. (위로 올린다.)
* 그 묶음 내에서 case id와 검사일자순으로 ascending sort


## 뒷부분: 나머지사람들
* case id 검사일로 소팅

In [433]:
df_valid = df_valid.sort_values(['case_id', 'event_time'])
head = df_valid[df_valid.contains_kw]
tail = df_valid[~df_valid.contains_kw]

In [434]:
head_case_ids = head.case_id.unique()
bring_to_head = tail.case_id.apply(lambda x: x in head_case_ids)

In [435]:
len(head.case_id.unique()), len(pd.concat([head, tail[bring_to_head]], axis=0).case_id.unique())

(1058, 1058)

In [438]:
head = pd.concat([head, tail[bring_to_head]]).sort_values(['case_id', 'event_time'])

In [439]:
tail = tail[~bring_to_head]

In [440]:
tail.shape

(1636, 9)

In [441]:
head.shape

(3242, 9)

In [442]:
df_valid.shape

(4878, 9)

In [443]:
tail.shape[0] + head.shape[0]

4878

In [445]:
pd.concat([head, tail]).to_csv("../data/interim/mri_readings_sorted.csv", index=False)

In [446]:
head.columns

Index(['id', 'case_id', 'event_type', 'event_name', 'event_time', 'event_result', 'contains_which_kw', 'contains_kw_not_negated', 'contains_kw'], dtype='object')

In [39]:
sorted_df = pd.read_csv("../data/interim/mri_readings_sorted.csv")

In [42]:
sorted_df.head()

,id,case_id,event_type,event_name,event_time,event_result,contains_which_kw,contains_kw_not_negated,contains_kw
0,945,17,IMG,MRI Brain + Brain MRA + Carotid MRA (contrast),2100-01-09 14:24:00,mild diffuse brain atrophy\nincreased both lat...,"['ischemi', 'infarc']",False,True
1,888,90,IMG,MRI Brain + MRA Acute Stroke (contrast),2100-01-04 04:29:00,"s/p craniotomy and sdh removal, lt. cerebral c...","['hemorrhage', 'hemorrhage']",True,True
2,864,216,IMG,MRA+MRI(Brain),2100-01-03 16:17:00,outside hospital mr on .\na small acute infarc...,['infarc'],True,True
3,468,846,IMG,CT Brain + 3D (noncontrast),2100-02-15 10:11:00,no evidence of large territorial infarction or...,"['hemorrhage', 'infarc', 'hemorrhage']",False,True
4,172,876,IMG,CT Routine Brain (Pre contrast),2100-01-17 06:39:00,interval disappeared previously noted sdh alon...,"['hemorrhage', 'hemorrhage']",False,True


In [37]:
clinical = clinical.rename(columns={'CaseID': 'case_id'})

In [46]:
sorted_df.merge(clinical[['case_id', 'Department']], how='inner', on='case_id').to_csv("../data/interim/mri_readings_sorted_with_dept.csv", index=False)

,id,case_id,event_type,event_name,event_time,event_result,contains_which_kw,contains_kw_not_negated,contains_kw,Department
0,945,17,IMG,MRI Brain + Brain MRA + Carotid MRA (contrast),2100-01-09 14:24:00,mild diffuse brain atrophy\nincreased both lat...,"['ischemi', 'infarc']",False,True,GS
1,888,90,IMG,MRI Brain + MRA Acute Stroke (contrast),2100-01-04 04:29:00,"s/p craniotomy and sdh removal, lt. cerebral c...","['hemorrhage', 'hemorrhage']",True,True,GS
2,864,216,IMG,MRA+MRI(Brain),2100-01-03 16:17:00,outside hospital mr on .\na small acute infarc...,['infarc'],True,True,GS
3,468,846,IMG,CT Brain + 3D (noncontrast),2100-02-15 10:11:00,no evidence of large territorial infarction or...,"['hemorrhage', 'infarc', 'hemorrhage']",False,True,GS
4,172,876,IMG,CT Routine Brain (Pre contrast),2100-01-17 06:39:00,interval disappeared previously noted sdh alon...,"['hemorrhage', 'hemorrhage']",False,True,GS
...,...,...,...,...,...,...,...,...,...,...
4873,14567,41284,IMG,CT Routine Brain (Pre contrast),2100-01-02 06:21:00,"s/p burr hole trephination and sdh removal, lt...",NaN,False,False,NS
4874,14641,41427,IMG,MRI Brain + Brain MRA + Carotid MRA (noncontrast),2100-01-02 16:07:00,[c.i.]\nmemory decline\n1. diffuse brain atrop...,NaN,False,False,GS
4875,14465,41842,IMG,MRI Brain + Brain MRA + Carotid MRA (contrast),2100-01-04 09:21:00,[c.i.]\nsyncope\n1. no abnormal focal lesion i...,NaN,False,False,PS
4876,14575,42218,IMG,MRI Brain Metastasis + DWI (contrast),2100-01-03 13:19:00,"[c.i.]\nr/o lung cancer, headache\nno evidence...",NaN,False,False,TS


# 코드 정리

In [6]:
raw_dir = '../data/raw'

In [1]:
keywords = """ischemi
infarc
high-intensity lesions on DWI-low intensity lesion on ADC
diffusion restriction lesion 
stroke
T2 high signal intensity 
low attenuation-GRE
hemorrhagic transformation
ICH
hemorrhage""".lower().split('\n')

negating_words = ['no', 'old', 'known', 'previous']

In [18]:
df = pd.read_csv(os.path.join(raw_dir, "lab.csv"), parse_dates=['event_time'], nrows=100)

In [22]:
df_valid.event_result.iloc[:30].tolist()

['mild diffuse brain atrophy\nincreased both lateral and 3rd ventricle size without visible obstructive lesion\nmild degree svd with a few old lacunar infarct in cerebral wm\nold ischemic change in rt prefrontal gyrus\nmra> \nrt proximal ica stenosis\nlt proximal',
 's/p craniotomy and sdh removal, lt. cerebral convexity.\n \nno deifnite evidence of diffusion restrictive lesion in the brain.\nminimal amount of small late subacute extraaxial hemorrhage (t1 and t2 high si) at lt occipital convexity.\na few microbleeds in th',
 'brain ct and cta\n1. no significant focal parenchymal lesion in the brain.\n    no significant abnormal finding at csf space. \n2. no significant focal stenosis or aneurysmal dilatation. \n3. emphysema and bullae at bilateral upper lungs.',
 'outside hospital mr on .\na small acute infarction in rt caudate body, and adjacent cr.\n- mild t2 high si.\n- no hemorrhagic changes.\nseveral old lacunar infarctions in both bg and rt thalamus.\nno abnormal contrast enhance

In [8]:
df = pd.read_csv(os.path.join(raw_dir, "lab.csv"), parse_dates=['event_time'])

# Time range
cutoff_start = pd.datetime(year=2100, month=1, day=1, hour=0, minute=0,)
cutoff_end = pd.datetime(year=2100, month=4, day=1, hour=0, minute=0,)

# Conditions
is_postop = (df.event_time > cutoff_start) & (df.event_time <= cutoff_end) 
is_img = df.event_type == 'IMG'
is_stroke_related = df.event_name.apply(lambda x: 'CT' in x or 'MR' in x)

# Filter data
df_valid = df[is_postop & is_img & is_stroke_related]

# Delete original df (too big)
del df

In [9]:
df_valid.loc[:, 'event_result'] = df_valid['event_result'].apply(lambda x: x.lower())

In [10]:
def contains_which_keyword(doc):
    has = []
    for kw in keywords:
        if kw in doc:
            if kw == 'stroke':
                if 'brain mri acute stroke' not in doc:
                    has.append(kw)
            else:
                has.append(kw)
    if len(has) == 0:
        return None
    else:
        return has

def contains_which_keyword_not_negated(doc):
    doc = doc.split('\n')
    for i, line in enumerate(doc):
        for kw in keywords:
            if kw in line:
                has_any_negated_kw = False
                for nw in negating_words:
                    match = re.search(
                        r'(?<={negation} ).*{keyword}'.format(negation=nw, keyword=kw), 
                        line
                    )
                    has_any_negated_kw |= match is not None

                if not has_any_negated_kw and 'brain mri acute stroke' not in line:
                    return True
    return False

In [11]:
df_valid.loc[:, 'contains_which_kw'] = df_valid['event_result'].apply(contains_which_keyword)
df_valid.loc[:, 'contains_kw_not_negated'] = df_valid['event_result'].apply(contains_which_keyword_not_negated)
df_valid['contains_kw'] = df_valid['contains_which_kw'].apply(lambda x: x is not None)

## Save readings with filters

In [ ]:
df_valid.to_csv("../../data/interim/filtered_readings.csv", index=False)

## Convert to binary label

Using string

In [ ]:
labels = df_valid[['case_id', 'contains_kw_not_negated']].groupby('case_id').sum().reset_index()
labels = labels.rename({'case_id': 'CaseID', 'contains_kw_not_negated': 'label'})
labels = (labels['label'] > 0).astype(int)